In [16]:
import polars as pl

#!uv pip install fastexcel 
#!uv pip install pyarrow

# Load the Excel file
df = pl.read_excel(
    "WUP2018-F22-Cities_Over_300K_Annual.xls", 
    read_options={"skip_rows": 16},  # Skip rows up to the header
    has_header=False,  # Indicate no header is currently in use
)

# from https://population.un.org/wup/downloads 
# -> Urban Agglomerations
# -> File 22: Annual Population of Urban Agglomerations with 300,000 Inhabitants or More in 2018, by country, 1950-2035 (thousands)


# Use the first row as headers and drop it
df.columns = [str(x) for x in df.row(0)]
df = df[1:]

# Rename floatlike column names such as "1950.0" → "1950"
df.columns = [
    str(int(float(col))) if col.endswith(".xx0") and col.replace(".0", "").isdigit() else col 
    for col in df.columns
]

# Cast Index, Country Code, City Code to integers; Latitude & Longitude to strings
df = df.with_columns(
    pl.col("Index").cast(pl.Int64),
    pl.col("Country Code").cast(pl.Int64),
    pl.col("City Code").cast(pl.Int64),
    pl.col("Latitude").cast(pl.Float64),
    pl.col("Longitude").cast(pl.Float64),
)

df.head()

Index,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,…,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
i64,i64,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,4,"""Afghanistan""",20001,"""Herat""",null,34.34817,62.19967,82.468,83.114,83.767,84.422,85.084,85.751,86.424,87.101,87.783,88.472,89.166,89.864,90.568,91.278,91.995,92.715,93.441,94.878,97.063,99.292,101.576,103.912,107.154,111.371,115.76,120.323,125.071,129.994,135.117,…,228.361,233.991,239.744,245.647,251.695,261.559,275.678,290.58,306.287,322.868,340.296,358.691,378.081,398.548,420.061,442.768,466.703,491.967,518.599,546.672,575.806,605.575,635.579,665.477,695.043,724.172,752.91,781.38,809.8,838.394,867.422,897.041,927.393,958.549,990.635,1023.636,1057.573
2,4,"""Afghanistan""",20002,"""Kabul""",null,34.528887,69.17246,170.784,179.779,189.26,199.213,209.705,220.749,232.391,244.613,257.496,271.057,285.352,300.359,316.177,332.829,350.382,368.809,388.232,408.087,428.363,449.585,471.891,495.303,528.508,573.161,621.656,674.254,731.385,793.178,860.289,…,2298.059,2401.109,2508.477,2620.804,2738.161,2834.061,2905.178,2978.181,3053.018,3129.844,3208.383,3289.005,3371.653,3456.496,3543.232,3632.269,3723.543,3817.241,3913.297,4011.77,4114.03,4221.532,4335.77,4457.882,4588.666,4728.384,4877.024,5034.106,5199.155,5371.55,5551.063,5737.138,5929.531,6127.953,6332.756,6543.594,6760.5
3,4,"""Afghanistan""",20003,"""Kandahar""",null,31.61332,65.71013,82.199,83.663,85.155,86.67,88.214,89.785,91.387,93.012,94.669,96.355,98.074,99.818,101.596,103.406,105.25,107.123,109.031,112.297,117.055,122.002,127.164,132.545,137.853,143.036,148.423,154.012,159.819,165.829,172.074,…,290.3,297.456,304.767,312.268,319.954,328.115,336.746,345.617,354.721,364.078,373.655,383.498,393.6,403.983,414.61,425.532,436.741,448.262,460.087,472.224,484.812,498.002,511.949,526.775,542.565,559.349,577.128,595.853,615.48,635.944,657.23,679.278,702.066,725.563,749.814,774.779,800.461
4,4,"""Afghanistan""",20004,"""Mazar-e Sharif""",null,36.70904,67.11087,30.0,31.308,32.676,34.099,35.586,37.139,38.761,40.449,42.213,44.054,45.979,47.981,50.074,52.258,54.541,56.916,59.399,61.989,64.697,67.515,70.46,73.533,76.745,80.087,83.58,87.226,91.036,95.001,99.144,…,168.224,172.372,176.61,180.958,185.413,193.721,206.403,219.935,234.353,249.739,266.088,283.532,302.12,321.955,343.032,365.52,389.483,415.053,442.302,471.339,501.665,532.689,563.825,594.551,624.5,653.486,681.531,708.798,735.572,762.162,788.909,816.04,843.747,872.137,901.348,931.381,962.262
5,8,"""Albania""",20005,"""Tiranë (Tirana)""",null,41.3275,19.81889,84.513,88.866,93.08,97.481,102.097,106.932,112.001,117.295,122.847,128.662,134.761,137.714,139.561,141.434,143.334,145.255,147.203,149.178,151.183,153.497,156.734,160.04,163.42,166.862,170.381,173.975,177.649,181.391,185.217,…,325.262,335.336,344.7,351.285,357.995,364.843,371.803,378.905,386.143,393.529,401.036,408.697,416.504,424.471,432.59,440.865,449.298,457.892,466.651,475.577,484.624,493.712,502.734,511.559,520.059,528.127,535.702,542.74,549.219,555.132,560.491,565.301,569.577,573.332,576.588,579.349,581.626


In [17]:
from csv_importer.parsers import polars_df_to_bob

In [18]:
from io import StringIO
import polars as pl
bob = polars_df_to_bob(df, name="World")

In [19]:
import bpy

# Load the node group from the blend file
with bpy.data.libraries.load("geonode_world_pop.blend", link=False) as (data_from, data_to):
    data_to.node_groups = [ng for ng in data_from.node_groups if ng == "world_population"]

# Assign the node group to the Geometry Nodes modifier
modifier = bob.object.modifiers.new(name="GeometryNodes", type='NODES')
modifier.node_group = bpy.data.node_groups["world_population"]

bpy.data.objects['World']

In [ ]:
import bpy
# Define the geographic bounding box for the entire Earth
west = -180.0  # Westernmost longitude
east = 180.0   # Easternmost longitude
north = 90.0   # Northernmost latitude
south = -90.0  # Southernmost latitude
# Calculate the center coordinates of the bounding box
center_x = (west + east) / 2
center_y = (north + south) / 2

map_location = (center_x, center_y , 0 )
# Add the plane and set it at the calculated center location
bpy.ops.mesh.primitive_plane_add(size=1, location= map_location)
plane = bpy.context.object

# Create a new material and assign the image texture
material = bpy.data.materials.new(name="ImageMaterial")
material.use_nodes = True
bsdf = material.node_tree.nodes["Principled BSDF"]

# Add and load the image texture
tex_image = material.node_tree.nodes.new('ShaderNodeTexImage')

import pathlib
image_location = pathlib.Path.cwd() / "Equirectangular_projection_SW.jpg"
tex_image.image = bpy.data.images.load(str(image_location))

# Connect the texture to the Base Color of the BSDF shader
material.node_tree.links.new(bsdf.inputs['Base Color'], tex_image.outputs['Color'])
plane.data.materials.append(material)

# Calculate the scale based on the geographic bounds
plane.scale.x = (east - west)
plane.scale.y = (north - south) 


: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Convert the Polars DataFrame to a pandas DataFrame for compatibility with matplotlib
df_pandas = df.to_pandas()

# Extract latitude, longitude, and 1965 data
latitudes = df_pandas["Latitude"]
longitudes = df_pandas["Longitude"]
values_1965 = df_pandas["1965"]

# Apply a logarithmic transformation to the 1965 data
log_values = np.log10(values_1965 + 1)  # Add 1 to avoid log(0)

# Scale the size of the dots proportionally to the population
dot_sizes = (log_values - log_values.min()) / (log_values.max() - log_values.min()) * 500 + 10  # Scale between 10 and 500

# Create a scatter plot
plt.figure(figsize=(12, 8))
scatter = plt.scatter(longitudes, latitudes, c=log_values, cmap="viridis", s=dot_sizes, edgecolor="k")

# Add a colorbar to represent the heatmap
colorbar = plt.colorbar(scatter, label="Log(Population (thousands) + 1) in 1965")

# Add labels and title
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap of Population (Log Scale, 1965)")

# Automatically adjust the axis limits to fit the data
plt.xlim(longitudes.min() - 2, longitudes.max() + 2)  # Add some padding around the min and max longitude
plt.ylim(latitudes.min() - 2, latitudes.max() + 2)  # Add some padding around the min and max latitude

# Show the plot
plt.show()

In [ ]:
# import subprocess
# import sys

# # Function to install a package
# def install_package(package_name):
#     try:
#         # Use subprocess to call pip
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
#         print(f"'{package_name}' installed successfully.")
#     except Exception as e:
#         print(f"Failed to install '{package_name}': {e}")

# # Install fastexcel
# install_package("pyarrow")
# install_package("fastexcel")